In [4]:
# importing libraries
import nltk
nltk.download('punkt')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet
nltk.download('wordnet')

import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alyonavishnoi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alyonavishnoi/nltk_data...


In [5]:
words=[]
classes = []
documents = []
ignore = ['?', '!','@','#','%','*','(',')','/']
data = open('intents.json').read()
intents = json.loads(data)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
# serialize the list
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))


# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
#represents the output of the neural network during training for each class/intent.

293 documents
128 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [6]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

The end result is a list of training samples, where each sample consists of a bag of words and the corresponding expected output row. These training samples can be used to train a neural network to recognize the intents based on the input patterns. The bag of words representation is a simple way to convert text data into a format that can be understood by the neural network.

In [7]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
#train_x will be used as the input features for training the neural network.
#train_y will be used as the target labels to train the neural network to recognize the correct intents based on the input patterns.
print("Training data created")

Training data created


/var/folders/b5/89nc1ktx2q3811mbrt9q9lrh0000gn/T/ipykernel_9024/951190035.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
#Dropout helps the model generalize better and reduces the risk of memorizing the training data.
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


this neural network architecture consists of three layers: two hidden layers with ReLU activation and dropout regularization, and one output layer with softmax activation. The dropout layers help improve generalization, and the softmax output layer provides the probabilities for different intent classes, allowing the model to make predictions for intent classification.

In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
import tensorflow as tf
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/Users/alyonavishnoi/anaconda3/lib/python3.10/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
59/59 [==============================] - 1s 3ms/step - loss: 4.8482 - accuracy: 0.0205
Epoch 2/200
59/59 [==============================] - 0s 4ms/step - loss: 4.7718 - accuracy: 0.0444
Epoch 3/200
59/59 [==============================] - 0s 2ms/step - loss: 4.6773 - accuracy: 0.0444
Epoch 4/200
59/59 [==============================] - 0s 2ms/step - loss: 4.5674 - accuracy: 0.0648
Epoch 5/200
59/59 [==============================] - 0s 3ms/step - loss: 4.4568 - accuracy: 0.0512
Epoch 6/200
59/59 [==============================] - 0s 2ms/step - loss: 4.3338 - accuracy: 0.0614
Epoch 7/200
59/59 [==============================] - 0s 3ms/step - loss: 4.2207 - accuracy: 0.0887
Epoch 8/200
59/59 [==============================] - 0s 3ms/step - loss: 4.0725 - accuracy: 0.0956
Epoch 9/200
59/59 [==============================] - 0s 3ms/step - loss: 3.9868 - accuracy: 0.0990
Epoch 10/200
59/59 [==============================] - 0s 3ms/step - loss: 3.8031 - accuracy: 0.1024
Epoch 11/

59/59 [==============================] - 0s 2ms/step - loss: 0.7900 - accuracy: 0.7645
Epoch 84/200
59/59 [==============================] - 0s 3ms/step - loss: 0.7525 - accuracy: 0.7850
Epoch 85/200
59/59 [==============================] - 0s 2ms/step - loss: 0.9531 - accuracy: 0.7167
Epoch 86/200
59/59 [==============================] - 0s 2ms/step - loss: 0.9786 - accuracy: 0.7201
Epoch 87/200
59/59 [==============================] - 0s 2ms/step - loss: 0.9378 - accuracy: 0.7235
Epoch 88/200
59/59 [==============================] - 0s 2ms/step - loss: 0.7446 - accuracy: 0.7918
Epoch 89/200
59/59 [==============================] - 0s 2ms/step - loss: 0.9838 - accuracy: 0.7235
Epoch 90/200
59/59 [==============================] - 0s 2ms/step - loss: 0.8914 - accuracy: 0.7577
Epoch 91/200
59/59 [==============================] - 0s 2ms/step - loss: 0.8168 - accuracy: 0.7577
Epoch 92/200
59/59 [==============================] - 0s 2ms/step - loss: 0.9976 - accuracy: 0.6826
Epoch 93/200


59/59 [==============================] - 0s 2ms/step - loss: 0.7851 - accuracy: 0.7850
Epoch 165/200
59/59 [==============================] - 0s 2ms/step - loss: 0.6971 - accuracy: 0.8225
Epoch 166/200
59/59 [==============================] - 0s 2ms/step - loss: 0.8343 - accuracy: 0.7509
Epoch 167/200
59/59 [==============================] - 0s 2ms/step - loss: 0.7602 - accuracy: 0.7816
Epoch 168/200
59/59 [==============================] - 0s 2ms/step - loss: 0.6562 - accuracy: 0.7884
Epoch 169/200
59/59 [==============================] - 0s 2ms/step - loss: 0.7761 - accuracy: 0.7713
Epoch 170/200
59/59 [==============================] - 0s 2ms/step - loss: 0.8219 - accuracy: 0.7713
Epoch 171/200
59/59 [==============================] - 0s 2ms/step - loss: 0.6584 - accuracy: 0.7918
Epoch 172/200
59/59 [==============================] - 0s 2ms/step - loss: 0.7036 - accuracy: 0.7747
Epoch 173/200
59/59 [==============================] - 0s 2ms/step - loss: 0.7109 - accuracy: 0.7543
Epoc

/Users/alyonavishnoi/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
